# JS - PLS + Gradient Boosted Trees

In [ ]:
import joblib
import json
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cross_decomposition import PLSRegression
from sklearn.impute import SimpleImputer
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import xgboost as xgb

In [ ]:
# read data as 32 bit floats
file = os.path.join(os.pardir, "input", "jane-street-market-prediction", "train.csv")
dtype = {c: np.float32 for c in pd.read_csv(file, nrows=1).columns}
full_df = pd.read_csv(file, engine="c", dtype=dtype)

# split into training and validation
train_df = full_df[full_df["date"].between(86, 375)]
valid_df = full_df[full_df["date"].between(425, 500)]

# build features and labels
features = [f"feature_{x}" for x in range(130)]
train_X = train_df[features].to_numpy()
valid_X = valid_df[features].to_numpy()
train_y = train_df["resp"].gt(0.0).astype(np.float32).to_numpy()
valid_y = valid_df["resp"].gt(0.0).astype(np.float32).to_numpy()

In [ ]:
# impute and normalize data
si = SimpleImputer(strategy="mean")
ss = StandardScaler()

train_X = si.fit_transform(train_X)
train_X = ss.fit_transform(train_X)

valid_X = si.transform(valid_X)
valid_X = ss.transform(valid_X)

# compute R2 scores of PLS model for different
# number of PLS components
#train_scores = np.empty(130)
#valid_scores = np.empty(130)
#components = np.arange(1, 131)
#for ix, comp in enumerate(components):
#    pls = PLSRegression(n_components=comp)
#    pls.fit(train_X, train_df["resp"])
#    train_scores[ix] = pls.score(train_X, train_df["resp"])
#    valid_scores[ix] = pls.score(valid_X, valid_df["resp"])

# plot the scores
#plt.figure(figsize=(8, 5))
#sns.lineplot(x=components, y=train_scores, label="Training")
#sns.lineplot(x=components, y=valid_scores, label="Validation")
#plt.xlabel("Components")    
#plt.ylabel("Score")
#plt.grid(True)
#plt.show()

# refit the model on the number of components a little
# after the elbow and transform the data
pls = PLSRegression(n_components=40)
pls.fit(train_X, train_df["resp"])
train_X = pls.transform(train_X)
valid_X = pls.transform(valid_X)

# save preprocessing pipeline
pp = make_pipeline(si, ss, pls)

In [ ]:
# parameters to optimize on validation set
params = {"max_depth": [7, 9, 11],
          "learning_rate": [0.05, 0.1, 0.15],
          "subsample": [0.6, 0.8, 0.9],
          "colsample_bytree": [0.5, 0.6, 0.7, 0.8],
          # "reg_alpha": [20.0],
          #"reg_lambda": [1.0]
         }

# early stopping
fit_params = {"early_stopping_rounds": 10,
              "eval_metric": "auc",
              "eval_set": [(valid_X, valid_y)]}

# base model
clf = xgb.XGBClassifier(n_estimators=1000, tree_method="gpu_hist", random_state=13)

# setup the data for parameter search
full_X = np.vstack([train_X, valid_X])
full_y = np.append(train_y, valid_y)
test_fold = np.append(np.full(train_X.shape[0], -1, dtype=np.int), np.zeros(valid_X.shape[0], dtype=np.int))
ps = PredefinedSplit(test_fold)

# find the parameters with best AUC score
cv = GridSearchCV(clf, params, cv=ps, scoring="roc_auc", refit=False)
cv.fit(full_X, full_y, **fit_params)

# show the best parameters
print(f"Highest AUC = {round(cv.best_score_, 4)} with parameters:")
for k, v in cv.best_params_.items():
    print(f"  {k} = {v}")

# need to refit the best model manually params, since the
# built-in refitting will refit on full_X, full_y
model = xgb.XGBClassifier(n_estimators=1000, tree_method="gpu_hist", random_state=13, **cv.best_params_)
model.fit(train_X, train_y, **fit_params)

# finally refit the model with the best number of estimators
model.set_params(n_estimators=clf.best_iteration)
model.fit(train_X, train_y)

In [ ]:
probs = model.predict_proba(valid_X)
probs = probs[:, 1]

# precision vs recall
precisions, recalls, thresholds = precision_recall_curve(valid_y, probs)

plt.figure(figsize=(8, 5))
plt.plot(thresholds, precisions[:-1], "tab:blue", label="Precision")
plt.plot(thresholds, recalls[:-1], "tab:orange", label="Recall")
plt.legend()
plt.xlabel("Threshold")
plt.title("Validation Precision/recall at threshold")
plt.axis([0, 1, 0, 1])
plt.grid(True)
plt.show()

plt.figure(figsize=(8, 5))
plt.plot(recalls, precisions, "tab:blue")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Validation  Precision at recall")
plt.axis([0, 1, 0, 1])
plt.grid(True)
plt.show()

# ROC curve
false_positives, true_positives, thresholds = roc_curve(valid_y, probs)
plt.figure(figsize=(8, 5))
plt.plot(false_positives, true_positives, "tab:blue")
plt.plot([0, 1], [0, 1], "tab:gray")
plt.xlabel("False positive rate")
plt.ylabel("True positive rate")
plt.title("Validation ROC curve")
plt.axis([0, 1, 0, 1])
plt.grid(True)
plt.show()

# AUC
print(f"Validation AUC: {roc_auc_score(valid_y, probs)}")

In [ ]:
probs = model.predict_proba(train_X)
probs = probs[:, 1]

# precision vs recall
precisions, recalls, thresholds = precision_recall_curve(train_y, probs)

plt.figure(figsize=(8, 5))
plt.plot(thresholds, precisions[:-1], "tab:blue", label="Precision")
plt.plot(thresholds, recalls[:-1], "tab:orange", label="Recall")
plt.legend()
plt.xlabel("Threshold")
plt.title("Training Precision/recall at threshold")
plt.axis([0, 1, 0, 1])
plt.grid(True)
plt.show()

plt.figure(figsize=(8, 5))
plt.plot(recalls, precisions, "tab:blue")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Training  Precision at recall")
plt.axis([0, 1, 0, 1])
plt.grid(True)
plt.show()

# ROC curve
false_positives, true_positives, thresholds = roc_curve(train_y, probs)
plt.figure(figsize=(8, 5))
plt.plot(false_positives, true_positives, "tab:blue")
plt.plot([0, 1], [0, 1], "tab:gray")
plt.xlabel("False positive rate")
plt.ylabel("True positive rate")
plt.title("Training ROC curve")
plt.axis([0, 1, 0, 1])
plt.grid(True)
plt.show()

# AUC
print(f"Training AUC: {roc_auc_score(train_y, probs)}")

In [ ]:
# save model
joblib.dump(pp, "preprocessor.pkl")
model.save_model("model.xgb")